# Dimensionality Reduction Using Feature Selection

## Seleção de Features: Uma Abordagem para Redução de Dimensionalidade  

A **seleção de features** consiste em escolher apenas as features mais relevantes e descartar as menos úteis.  

### Tipos de Seleção de Features  

Existem **três métodos principais** para selecionar features:  

1. **Métodos de Filtro**  
   - Escolhem as melhores features analisando suas propriedades estatísticas.  

2. **Métodos Wrapper**  
   - Testam diferentes combinações de features para encontrar o conjunto que gera o melhor desempenho no modelo.  

3. **Métodos Embutidos (Embedded)**  
   - A seleção de features acontece como parte do treinamento de um algoritmo de aprendizado de máquina.  


## 1. Variance Thresholding

Remove colunas (features) que tem variação muito baixa, ou seja, imagina que tem um monte de colunas em uma tabela cheia de números e quer tirar as que quase não mudam, então remove as com variância baixa

Com isso, calculamos a variância de cada coluna, se os valores de uma coluna mudam bastante então a variância é alta. Depois definimos um limite e removemos as colunas com a variância menor que esse limite.

Aviso: não funciona bem se os dados estiverem em escalas diferentes, porém se padronizar tudo também não será uma técnica útil

In [1]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

features = iris.data
target = iris.target

thresholder = VarianceThreshold(threshold=.5) # variância menor que 0.5

features_high_variance = thresholder.fit_transform(features) # aplicamos o filtro que remove as colunas com variância baixa

features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [2]:
thresholder.fit(features).variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

## 2. Thresholding Binary Feature Variance

Técnica usada quando temos um monte de colunas com valores binários (0 ou 1) e queremos remover aquelas que quase não mudam.

Deve ser usado quando:
- Se tem muitas colunas binárias (0s e 1s) e quer remover as que são quase sempre iguais.
- Se os dados têm muitas features desbalanceadas (exemplo: uma coluna que tem 99% de 0s).

In [3]:
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

thresholder = VarianceThreshold(threshold=(0.75 * (1 - 0.75)))

features_filtradas = thresholder.fit_transform(features)
features_filtradas

array([[0],
       [1],
       [0],
       [1],
       [0]])

## 3. Handling Highly Correlated Features

Quando tem um conjunto de dados com várias colunas (features) e suspeita que algumas delas sejam muito parecidas (ou seja, são altamente correlacionadas). Por isso é necessário descobrir quais colunas são muito parecidas e remover uma delas.

Para isso:
- Criamos uma matriz de correlação
- Procura correlações muito altas
- Remove uma das colunas correlacionadas

In [6]:
import pandas as pd
import numpy as np

features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

df = pd.DataFrame(features)
df

,0,1,2
0,1,1,1
1,2,2,0
2,3,3,1
3,4,4,0
4,5,5,1
5,6,6,0
6,7,7,1
7,8,7,0
8,9,7,1


In [7]:
corr_matriz = df.corr().abs() # pega os valores absolutos 

# usar somente a parte superior da matriz para não repetir os pares
upper = corr_matriz.where(np.triu(np.ones(corr_matriz.shape), k=1).astype(bool))

# encontramos as colunas com correlação maior que 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

df_filtrado = df.drop(df.columns[to_drop], axis=1)
df_filtrado

,0,2
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1


## 4. Removing Irrelevant Features for Classification

Usamos quando temos um conjunto de dados com várias colunas (features) e um alvo (target) que queremos prever. Mas algumas dessas colunas podem ser irrelevantes para a previsão. Por isso escolhemos as mais úteis usando testes estatísticos que medem a relação entre as colunas e o target (veêm quais tem mais relações com a target).

- Qui-quadrado (χ²) → Para colunas categóricas
- F-Valor (ANOVA F-test) → Para colunas numéricas

In [8]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2

iris = load_iris()
features = iris.data
target = iris.target

# transforma dados para inteiros pois chi² precisa de valores positivos
features = features.astype(int)

# selecionamos as 2 colunas mais relevantes com chi²
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

print("Número original de colunas:", features.shape[1])
print("Número reduzido de colunas:", features_kbest.shape[1])

Número original de colunas: 4
Número reduzido de colunas: 2


In [9]:
from sklearn.feature_selection import f_classif

# selecionamos as 2 colunas mais relevantes usando ANOVA F-test
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print("Número original de colunas:", features.shape[1])
print("Número reduzido de colunas:", features_kbest.shape[1])

Número original de colunas: 4
Número reduzido de colunas: 2


Se não sabe quantas colunas manter → Em vez de SelectKBest(k=2), pode usar SelectPercentile(percentile=75), que mantém as melhores 75% das colunas

**Por fim, se uma coluna não ajuda a prever o target, ela é removida para deixar o modelo mais eficiente**

## 5. Recursively Eliminating Features

O RFE escolhe as features mais importantes para o modelo automaticamente. Ele usa validação cruzada para garantir que a remoção de variáveis não degrade a performance. Quanto menor o número de variáveis relevantes, mais eficiente fica o modelo.


In [10]:
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model


warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

features, target = make_regression(n_samples = 10000, n_features = 100, n_informative = 2, random_state = 1)

ols = linear_model.LinearRegression()

rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.0992611 ,  0.7031277 , -0.75164317, -1.20934598],
       [-1.07500204,  0.92859616,  2.56148527, -1.68392493, -1.73223805],
       [ 1.37940721,  1.83471056, -1.77039484,  0.12954322,  0.03519776],
       ...,
       [-0.80331656, -0.40335314, -1.60648007,  0.20986659, -0.05626806],
       [ 0.39508844, -0.98395086, -1.34564911, -1.21942363, -1.50212012],
       [-0.55383035,  0.05065251,  0.82880112,  0.05486622,  0.64327968]],
      shape=(10000, 5))

In [11]:
# número de melhores features
rfecv.n_features_

np.int64(5)

In [12]:
rfecv.ranking_

array([67, 76, 71, 52, 69,  1, 95, 94, 78, 64, 11, 75, 70, 90, 14, 19, 20,
       77, 83,  1, 59, 21, 12,  9, 13, 39, 22, 10,  8, 93, 89, 62, 55, 73,
       45, 88, 72, 79, 26,  1,  1, 16,  6, 42, 28, 37,  2, 63, 96, 46, 86,
       32, 54, 65, 68, 91, 24, 57,  1, 84, 49, 53, 50, 60, 23, 87, 27, 30,
       36,  5, 29, 51, 17, 18, 34, 31,  3, 58, 43,  7, 44, 66, 80, 33, 15,
       85, 40, 61, 38, 56, 92, 74, 81, 35,  4, 25, 82, 48, 41, 47])